In [1]:
import sqlite3
import pandas as pd
import math
from scipy import stats
import seaborn as sns
import numpy as np

In [2]:
connection = sqlite3.connect('Northwind_small.sqlite')
cursor = connection.cursor()

def sql_select_to_df(SQL_COMMAND, cursor=cursor):
    results = cursor.execute(SQL_COMMAND).fetchall()
    df = pd.DataFrame(results)
    df.columns = [i[0] for i in cursor.description]
    return df

In [3]:
 product_orderDetails_df = sql_select_to_df("""SELECT *
                                FROM [Product] AS P
                                JOIN [OrderDetail] AS O
                                ON P.ID = O.ProductID
                        """)

product_orderDetails_df

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,11,Queso Cabrales,5,4,1 kg pkg.,21.00,22,30,30,0,10248/11,10248,11,14.00,12,0.00
1,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1,10248/42,10248,42,9.80,10,0.00
2,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,14,0,0,0,10248/72,10248,72,34.80,5,0.00
3,14,Tofu,6,7,40 - 100 g pkgs.,23.25,35,0,0,0,10249/14,10249,14,18.60,9,0.00
4,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10249/51,10249,51,42.40,40,0.00
5,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65,85,0,10,0,10250/41,10250,41,7.70,10,0.00
6,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,20,0,10,0,10250/51,10250,51,42.40,35,0.15
7,65,Louisiana Fiery Hot Pepper Sauce,2,2,32 - 8 oz bottles,21.05,76,0,0,0,10250/65,10250,65,16.80,15,0.15
8,22,Gustaf's Knäckebröd,9,5,24 - 500 g pkgs.,21.00,104,0,25,0,10251/22,10251,22,16.80,6,0.05
9,57,Ravioli Angelo,26,5,24 - 250 g pkgs.,19.50,36,0,20,0,10251/57,10251,57,15.60,15,0.05


In [23]:
category_IDs = product_orderDetails_df['CategoryId'].values
category_IDs = list(category_IDs)
prices = product_orderDetails_df.iloc[:,13].values
quantities = product_orderDetails_df['Quantity'].values

revenues = prices * quantities
list(revenues)

[168.0,
 98.0,
 174.0,
 167.4,
 1696.0,
 77.0,
 1484.0,
 252.0,
 100.80000000000001,
 234.0,
 336.0,
 2592.0,
 50.0,
 1088.0,
 200.0,
 604.8000000000001,
 640.0,
 54.0,
 403.2,
 168.0,
 304.0,
 486.5,
 380.0,
 1320.0,
 393.0,
 124.80000000000001,
 877.5,
 86.4,
 156.0,
 760.0,
 1105.0,
 153.60000000000002,
 80.0,
 20.8,
 123.2,
 780.0,
 591.0,
 252.0,
 160.0,
 288.0,
 204.0,
 360.0,
 60.8,
 834.0,
 100.8,
 1242.0,
 288.0,
 532.0,
 192.5,
 936.0,
 240.0,
 364.79999999999995,
 735.0,
 3080.0,
 216.0,
 990.0,
 111.2,
 120.0,
 556.0,
 456.0,
 919.9999999999999,
 48.0,
 388.79999999999995,
 400.0,
 667.2,
 595.2,
 150.0,
 40.0,
 882.0,
 475.2,
 344.0,
 194.6,
 43.2,
 264.0,
 372.0,
 48.0,
 728.0,
 472.79999999999995,
 248.0,
 660.0,
 280.8,
 300.0,
 468.0,
 43.2,
 384.0,
 186.0,
 7.3,
 21.6,
 57.6,
 124.19999999999999,
 31.2,
 248.0,
 131.4,
 952.0,
 83.4,
 526.5,
 325.5,
 544.0,
 56.0,
 648.0,
 588.0,
 943.1999999999999,
 1440.0,
 1576.0,
 556.0,
 224.0,
 144.0,
 59.0,
 30.0,
 240.0,
 239.

In [15]:
print(product_orderDetails_df['CategoryId'].unique())

[4 5 7 8 2 3 1 6]


In [30]:
def get_category_revenues_per_category(categoryNum, revenues_list, IDs_list):
    master_list = []
    iterator_count = 0
    
    for i in IDs_list:
        if i == categoryNum:
            master_list.append(revenues_list[iterator_count])
    
        iterator_count+=1
    
    return master_list
            

id4_revenues = get_category_revenues_per_category(4,revenues,category_IDs)
id5_revenues = get_category_revenues_per_category(5,revenues,category_IDs)
id7_revenues = get_category_revenues_per_category(7,revenues,category_IDs)
id8_revenues = get_category_revenues_per_category(8,revenues,category_IDs)
id2_revenues = get_category_revenues_per_category(2,revenues,category_IDs)
id3_revenues = get_category_revenues_per_category(3,revenues,category_IDs)
id1_revenues = get_category_revenues_per_category(1,revenues,category_IDs)
id6_revenues = get_category_revenues_per_category(6,revenues,category_IDs)

In [33]:
check1 = len(id4_revenues) + len(id5_revenues) + len(id7_revenues) + len(id8_revenues) + len(id2_revenues) + len(id3_revenues) + len(id1_revenues) + len(id6_revenues)
check2 = len(revenues)

check1 == check2

True

In [ ]:
def get_control_